In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [9]:
#收益率概率分布
def return_pro_dis(df, rolling_window):       #df:含['日期'、'涨跌幅(%)']字段的dataframe     rolling_window:窗口大小
    #数据预处理
    df.set_index('日期',inplace=True)
    df = df.sort_index()
    df.rename(columns={'涨跌幅(%)':'1日收益率(%)'},inplace=True)
    df['滚动收益率(%)'] = df['1日收益率(%)'].rolling(rolling_window).mean()   #n日滚动收益
    df.fillna(0,inplace=True)  #填充滚动收益率空值
    
    #收益率概率密度
    density = stats.kde.gaussian_kde(df['滚动收益率(%)'])  #高斯核估计PDE  详见https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gaussian_kde.html
    return density   #通过density.evaluate(point)可以得到某一收益率的概率

In [16]:
#成交量概率分布(1天rolling_window)
def turnover_pro_dis(df):       #df:含['日期'、'成交量(股)']字段的dataframe
    #成交量概率密度
    density = stats.kde.gaussian_kde(df['成交量(股)'])  #高斯核估计PDE
    return density   #通过density.evaluate(point)可以得到某一成交量的概率

##### 举例（收益率概率分布）

In [139]:
#参数
df = pd.read_excel(r'C:\Users\lenovo\Desktop\资料\暑期资料\量化\历史数据\600711.SH 涨跌幅.xlsx') #从同花顺导出日期、涨跌幅
rolling_window = 3

density = return_pro_dis(df, rolling_window)

#某一收益率的概率
print('该收益率下的概率：',density.evaluate(2))

#某一区间累计概率(需要手动插值找出分位数)
'''
经过手工插值,
600711 50%分位：(-10,-0.03)   60%分位：（-10,0.43）     70%分位：（-10,0.53)     80%分位：（-10,1.49）    90%分位：(-10,2.38)
       95%分位：(-10,3.28)    99%分位：（-10，5.37）
'''

print('区间累积概率：',density.integrate_box_1d(-10,5.37))  #参数：区间下限（收益率最低-10），区间上限



# #密度分布画图
# bins = np.arange(-10,10,0.01)
# plt.plot(bins,density(bins))
# plt.title('Return density')

该收益率下的概率： [0.10638466]
区间累积概率： 0.9899414273339797


##### 举例（成交量概率分布）

In [143]:
df = pd.read_excel(r'C:\Users\lenovo\Desktop\资料\暑期资料\量化\历史数据\600711.SH 成交量.xlsx') #从同花顺导出日期、成交量
density = turnover_pro_dis(df)

#某一成交量的概率
print('该成交量下的概率：',density.evaluate(80000000))
print('区间累积概率：',density.integrate_box_1d(0,86700000))

# #密度分布画图
# bins = np.arange(0,99999999,10000)
# plt.plot(bins,density(bins))
# plt.title('Turnover density')

该成交量下的概率： [7.55320644e-09]
区间累积概率： 0.5000780419294297
